# Machine Learning Project
Arthur Olivieri, David Conselvan and Pedro Stanzani

## Context

In this project, we aim to predict the Severity Impairment Index (sii), a measure of problematic internet use among adolescents, using data from the Healthy Brain Network (HBN) study. 

The dataset combines multi-modal information, including tabular data on:
- demographics;
- physical activity;
- fitness;
- sleep patterns; and
- internet usage

Our approach will involve thorough data preprocessing to handle missing values, normalize features, and extract meaningful summaries from the time-series data. We will perform exploratory data analysis (EDA) to understand the relationships between predictors and the target variable, followed by feature engineering to combine tabular and time-series data effectively. Using these features, we will build and evaluate machine learning models, such as ensemble models for tabular data and sequence models (e.g., RNNs or Transformers) for the accelerometer data. 

The integration of these models will enable us to develop a robust system for predicting problematic internet use, contributing insights to mental health research and potentially informing interventions for at-risk adolescents.

View also: https://www.kaggle.com/c/child-mind-institute-problematic-internet-use/

## Target

Note in particular the field PCIAT-PCIAT_Total. The target sii for this competition is derived from this field as described in the data dictionary: 0 for None, 1 for Mild, 2 for Moderate, and 3 for Severe. Additionally, each participant has been assigned a unique identifier id.

In [ ]:
import numpy as np

def PCIAT_PCIAT_Total_to_sii(PCIAT_PCIAT_Total):
    if np.isnan(PCIAT_PCIAT_Total):
        return np.nan
    elif 0 <= PCIAT_PCIAT_Total <= 30:
        return 0  # None
    elif 31 <= PCIAT_PCIAT_Total <= 49:
        return 1  # Mild
    elif 50 <= PCIAT_PCIAT_Total <= 79:
        return 2  # Moderate
    elif 80 <= PCIAT_PCIAT_Total <= 100:
        return 3  # Severe
    else:
        raise ValueError("Invalid PCIAT_PCIAT_Total value. It must be between 0 and 100 or NaN.")

def get_severity_level(score):
    if 0 <= score <= 30:
        return 'None'
    elif 31 <= score <= 49:
        return 'Mild'
    elif 50 <= score <= 79:
        return 'Moderate'
    elif 80 <= score <= 100:
        return 'Severe'
    else:
        return np.nan

## Setup

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer

## 1. Exploratory Data Analysis

In [ ]:
plt.style.use("ggplot")
sns.set_palette("muted")
%matplotlib inline

In [ ]:
train_data = pd.read_csv('../data/train.csv')
test_data = pd.read_csv('../data/test.csv')
data_dict = pd.read_csv('../data/data_dictionary.csv')

In [ ]:
train_data = train_data.dropna(subset=['sii'])

In [ ]:
target_related_columns = [col for col in train_data.columns if col not in test_data.columns]
target_related_columns

In [ ]:
train_data.head()

### 1.1. Column information

In [ ]:
train_data.info()

### 1.2. Missing values

In [ ]:
missing_values = train_data.isnull().sum()
missing_percentage = (missing_values / len(train_data)) * 100

print(missing_percentage[missing_percentage > 0])

In [ ]:
plt.figure(figsize=(12, 6))
sns.heatmap(train_data.isnull(), cbar=False, cmap='viridis')
plt.title("Missing Values Heatmap")
plt.show()

### 1.3.  Descriptive statistics

**Descriptive statistics (numerical data)**

In [ ]:
train_data.describe()

**Descriptive statistics (Categorical Data)**

In [ ]:
train_data.describe(include=['object', 'category'])

### 1.4. Analyze the target variable

In [ ]:
train_data['Severity_Level'] = train_data['PCIAT-PCIAT_Total'].apply(get_severity_level)
train_data['Severity_Level'].value_counts()

In [ ]:
# Bar plot of Severity Levels
sns.countplot(x='Severity_Level', data=train_data, order=['None', 'Mild', 'Moderate', 'Severe'])
plt.title('Distribution of Severity Levels')
plt.xlabel('Severity Level')
plt.ylabel('Count')
plt.show()

# Histogram of PCIAT_Total scores
sns.histplot(train_data['PCIAT-PCIAT_Total'], bins=30, kde=True)
plt.title('Distribution of PCIAT_Total Scores')
plt.xlabel('PCIAT_Total Score')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# Check for class imbalance
severity_counts = train_data['Severity_Level'].value_counts()
severity_counts / len(train_data)

### 1.5. Correlation analysis

In [ ]:

numerical_features = [
  "Basic_Demos-Age",
  "CGAS-CGAS_Score",
  "Physical-BMI",
  "Physical-Height",
  "Physical-Weight",
  "Physical-Waist_Circumference",
  "Physical-Diastolic_BP",
  "Physical-HeartRate",
  "Physical-Systolic_BP",
  "Fitness_Endurance-Time_Mins",
  "Fitness_Endurance-Time_Sec",
  "FGC-FGC_CU",
  "FGC-FGC_GSND",
  "FGC-FGC_GSD",
  "FGC-FGC_PU",
  "FGC-FGC_SRL",
  "FGC-FGC_SRR",
  "FGC-FGC_TL",
  "BIA-BIA_BMC",
  "BIA-BIA_BMI",
  "BIA-BIA_BMR",
  "BIA-BIA_DEE",
  "BIA-BIA_ECW",
  "BIA-BIA_FFM",
  "BIA-BIA_FFMI",
  "BIA-BIA_FMI",
  "BIA-BIA_Fat",
  "BIA-BIA_ICW",
  "BIA-BIA_LDM",
  "BIA-BIA_LST",
  "BIA-BIA_SMM",
  "BIA-BIA_TBW",
  "PAQ_A-PAQ_A_Total",
  "PAQ_C-PAQ_C_Total",
  # "PCIAT-PCIAT_Total",
  "SDS-SDS_Total_Raw",
  "SDS-SDS_Total_T",
]

# Visualize distributions using histograms
train_data[numerical_features].hist(bins=15, figsize=(15, 20), layout=(len(numerical_features) // 3 + 1, 3))
plt.tight_layout()
plt.show()

# Analyze correlations between numerical features and the target variable
corr_features = numerical_features + ['PCIAT-PCIAT_Total']
corr_data = train_data[corr_features]
correlation_matrix = corr_data.corr()
plt.figure(figsize=(15, 12))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix')
plt.show()

In [ ]:
correlation_matrix['PCIAT-PCIAT_Total'].abs().sort_values(ascending=False)

In [ ]:
# Remove features that may not be in the dataset
categorical_features = [
    "Basic_Demos-Enroll_Season",
    "CGAS-Season",
    "Physical-Season",
    "Fitness_Endurance-Season",
    "Fitness_Endurance-Max_Stage",
    "FGC-Season",
    "FGC-FGC_CU_Zone",
    "FGC-FGC_GSND_Zone",
    "FGC-FGC_GSD_Zone",
    "FGC-FGC_PU_Zone",
    "FGC-FGC_SRL_Zone",
    "FGC-FGC_SRR_Zone",
    "FGC-FGC_TL_Zone",
    "BIA-Season",
    "BIA-BIA_Activity_Level_num",
    "BIA-BIA_Frame_num",
    "PAQ_A-Season",
    "PAQ_C-Season",
    "SDS-Season",
    "PreInt_EduHx-Season",
    "PreInt_EduHx-computerinternet_hoursday",
    "Basic_Demos-Sex"
]

categorical_features = [feature for feature in categorical_features if feature in train_data.columns]

# Analyze the distribution of categories for key features
fig, axes = plt.subplots(nrows=(len(categorical_features) + 2) // 3, ncols=3, figsize=(18, 5 * ((len(categorical_features) + 2) // 3)))
axes = axes.flatten()

for idx, feature in enumerate(categorical_features):
    sns.countplot(x=feature, data=train_data, ax=axes[idx])
    axes[idx].set_title(f'Distribution of {feature}')
    axes[idx].set_xlabel(feature)
    axes[idx].set_ylabel('Count')
    axes[idx].tick_params(axis='x', rotation=45)

# Hide any unused subplots
for idx in range(len(categorical_features), len(axes)):
    axes[idx].axis('off')

plt.tight_layout()
plt.show()

# Visualize relationships between categorical features and the target variable
fig, axes = plt.subplots(nrows=(len(categorical_features) + 2) // 3, ncols=3, figsize=(18, 5 * ((len(categorical_features) + 2) // 3)))
axes = axes.flatten()

for idx, feature in enumerate(categorical_features):
    sns.countplot(x=feature, hue='Severity_Level', data=train_data, order=train_data[feature].value_counts().index, ax=axes[idx])
    axes[idx].set_title(f'{feature} vs Severity Level')
    axes[idx].set_xlabel(feature)
    axes[idx].set_ylabel('Count')
    axes[idx].legend(title='Severity Level')
    axes[idx].tick_params(axis='x', rotation=45)

# Hide any unused subplots
for idx in range(len(categorical_features), len(axes)):
    axes[idx].axis('off')

plt.tight_layout()
plt.show()

## 2. Preprocessing

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split

target_column = 'sii'
X = train_data[numerical_features + categorical_features]
y = train_data[target_column]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the column transformer
preprocessor = ColumnTransformer(
    
    transformers=[
        ('num', StandardScaler(), numerical_features),  # Standardize numerical features
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)  # One-hot encode categorical features
    ]
)

# Fit the transformer on the training data
preprocessor.fit(X_train)

# Transform both train and test data
X_train_transformed = preprocessor.transform(X_train)
X_test_transformed = preprocessor.transform(X_test)

# Output shapes
print(f"Transformed X_train shape: {X_train_transformed.shape}")
print(f"Transformed X_test shape: {X_test_transformed.shape}")

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('imputer', SimpleImputer(strategy='mean')),  # Handle NaN values
    ('dim_reduction', PCA()),
    ('classifier', RandomForestClassifier())  # Placeholder
])

pipeline

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier
from sklearn.naive_bayes import GaussianNB

# Models to use
models = {
    'LogisticRegression': LogisticRegression(max_iter=1000, random_state=42),
    'SVC': SVC(random_state=42),
    'RandomForest': RandomForestClassifier(random_state=42),
    'GaussianNB': GaussianNB(),
    'Dummy': DummyClassifier(strategy='most_frequent', random_state=42)
}

# Example: Setting the pipeline's classifier to one of these models
pipeline.set_params(classifier=models['LogisticRegression'])

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import GridSearchCV

# Define parameter grids
param_grids = [
    {
        'dim_reduction__n_components': [5, 10, 20, 50],
        'classifier': [SVC()],
        'classifier__C': [0.1, 1, 10],
        'classifier__kernel': ['linear', 'rbf']
    },
    {
        'dim_reduction__n_components': [5, 10, 20, 50],
        'classifier': [LogisticRegression(max_iter=1000, random_state=42)],
        'classifier__C': [0.1, 1, 10]
    },
    {
        'dim_reduction__n_components': [5, 10, 20, 50],
        'classifier': [RandomForestClassifier(random_state=42)],
        'classifier__n_estimators': [50, 100, 200],
        'classifier__max_depth': [None, 10, 20]
    },
    {
        'dim_reduction__n_components': [5, 10, 20, 50],
        'classifier': [GaussianNB()]

    }
]

# Use GridSearchCV with combined parameter grids
grid_search = GridSearchCV(pipeline, param_grids, cv=5, scoring='accuracy', verbose=2)

# Fit GridSearchCV
grid_search.fit(X_train, y_train)

dummy_clf = DummyClassifier(strategy='most_frequent')


In [ ]:
# Output best parameters and score
print("Best Parameters: ", grid_search.best_params_)
print("Best Score: ", grid_search.best_score_)
best_model = grid_search.best_estimator_
print(best_model)


### 3. Re-train best models and Deploy the Final Model

In [ ]:
from sklearn.metrics import classification_report, accuracy_score
target_names = ['None', 'Mild', 'Moderate', 'Severe']

from sklearn.utils.class_weight import compute_class_weight

# Balancing the classes
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
best_model.named_steps['classifier'].set_params(class_weight=dict(enumerate(class_weights)))

# Retrain the best model on the training dataset
best_model = grid_search.best_estimator_
best_model.fit(X, y)

# Retrieve the best model
best_model = grid_search.best_estimator_

# Predict on the test set
y_pred = best_model.predict(X_test)

# Evaluate the model
print("Classification Report for the Best Model:")
print(classification_report(y_test, y_pred, target_names= target_names))

# Evaluate the dummy classifier
dummy_clf.fit(X_train, y_train)
y_dummy_pred = dummy_clf.predict(X_test)
print("Classification Report for the Dummy Classifier:")
print(classification_report(y_test, y_dummy_pred, target_names= target_names))



### 4. Evaluation of Results: Addressing Class Imbalance and Model Performance


In [ ]:

from sklearn.metrics import balanced_accuracy_score

balanced_acc = balanced_accuracy_score(y_test, y_pred)
balanced_acc_dummy = balanced_accuracy_score(y_test, y_dummy_pred)

print("Normal accuracy Best Model",accuracy_score(y_test, y_pred))
print("Normal accuracy Dummy", accuracy_score(y_test, y_dummy_pred))
print("Balanced Accuracy Best Model:", balanced_acc)
print("Balanced Accuracy Dummy:", balanced_acc_dummy)


#### Overview
The dataset used in this analysis exhibits significant class imbalance. The majority class (None) accounts for ~58% of the data, while the Severe class represents only ~1%. This imbalance poses a challenge for the classifier, as models tend to favor predicting the majority class, leading to misleadingly high accuracy but poor performance on minority classes.

##### Metrics Analysis


- Normal Accuracy:

    - Best Model: 52.92% 


    - Dummy Classifier: 61.31%


The dummy classifier achieves higher accuracy by always predicting the majority class (None). However, this is not a meaningful success as it completely ignores minority classes.


The best model, while less accurate overall, attempts to balance predictions across classes, leading to lower accuracy but better generalization to minority classes.
Balanced Accuracy:

- Best Model: 53.64%


- Dummy Classifier: 25.00%


Balanced accuracy measures the average recall across all classes, giving equal importance to each. The best model significantly outperforms the dummy classifier here, showing its ability to make predictions across all classes, even minority ones.


The dummy classifier’s poor balanced accuracy reflects its complete failure to predict any minority classes (Mild, Moderate, Severe).

### 5. Predicting with the competition's test Dataset

In [ ]:
X_final = test_data[numerical_features + categorical_features]
y_final = best_model.predict(X_final)
y_final


In [ ]:
[col for col in test_data.columns if 'id' in col.lower()]

In [ ]:
s = ''
for i, row in test_data.iterrows():
    s += f'{row['id']},{int(y_final[i])}\n'

with open('submission.csv', 'w') as f:
    f.write('id,sii\n')
    f.write(s)
